# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 20 2022 8:28AM,253453,19,ACG-2102492187,ACG North America LLC,Released
1,Dec 20 2022 8:20AM,253452,19,ADV80009385,"AdvaGen Pharma, Ltd",Released
2,Dec 20 2022 8:20AM,253452,19,ADV80009386,"AdvaGen Pharma, Ltd",Released
3,Dec 20 2022 8:20AM,253452,19,ADV80009387,"AdvaGen Pharma, Ltd",Released
4,Dec 20 2022 8:20AM,253452,19,ADV80009388,"AdvaGen Pharma, Ltd",Released
5,Dec 20 2022 8:20AM,253452,19,ADV80009389,"AdvaGen Pharma, Ltd",Released
6,Dec 20 2022 8:20AM,253452,19,ADV80009391,"AdvaGen Pharma, Ltd",Released
7,Dec 20 2022 8:20AM,253452,19,ADV80009392,"AdvaGen Pharma, Ltd",Released
8,Dec 20 2022 8:20AM,253452,19,ADV80009393,"AdvaGen Pharma, Ltd",Released
9,Dec 20 2022 8:20AM,253452,19,ADV80009394,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,253446,Released,15
25,253448,Released,1
26,253449,Released,30
27,253452,Released,48
28,253453,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Released
id,
253446,15
253448,1
253449,30
253452,48
253453,1


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Released
id,
253377,1
253382,1
253383,1
253384,3
253387,3


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,Released
id,
253377,1
253382,1
253383,1
253384,3
253387,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Released
0,253377,1
1,253382,1
2,253383,1
3,253384,3
4,253387,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,id,Released
0,253377,1
1,253382,1
2,253383,1
3,253384,3
4,253387,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id          int64
Released    int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 20 2022 8:28AM,253453,19,ACG North America LLC
1,Dec 20 2022 8:20AM,253452,19,"AdvaGen Pharma, Ltd"
49,Dec 19 2022 4:58PM,253449,15,"Brookfield Pharmaceuticals, LLC"
79,Dec 19 2022 4:56PM,253448,15,"Virtus Pharmaceuticals, LLC"
80,Dec 19 2022 4:55PM,253446,15,"Alliance Pharma, Inc."
95,Dec 19 2022 4:50PM,253445,10,"Snap Medical Industries, LLC"
97,Dec 19 2022 4:48PM,253444,22,"NBTY Global, Inc."
98,Dec 19 2022 4:47PM,253442,10,ISDIN Corporation
111,Dec 19 2022 4:47PM,253441,10,ISDIN Corporation
121,Dec 19 2022 4:39PM,253440,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Released
0,Dec 20 2022 8:28AM,253453,19,ACG North America LLC,1
1,Dec 20 2022 8:20AM,253452,19,"AdvaGen Pharma, Ltd",48
2,Dec 19 2022 4:58PM,253449,15,"Brookfield Pharmaceuticals, LLC",30
3,Dec 19 2022 4:56PM,253448,15,"Virtus Pharmaceuticals, LLC",1
4,Dec 19 2022 4:55PM,253446,15,"Alliance Pharma, Inc.",15
5,Dec 19 2022 4:50PM,253445,10,"Snap Medical Industries, LLC",2
6,Dec 19 2022 4:48PM,253444,22,"NBTY Global, Inc.",1
7,Dec 19 2022 4:47PM,253442,10,ISDIN Corporation,13
8,Dec 19 2022 4:47PM,253441,10,ISDIN Corporation,10
9,Dec 19 2022 4:39PM,253440,10,ISDIN Corporation,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released
0,Dec 20 2022 8:28AM,253453,19,ACG North America LLC,1
1,Dec 20 2022 8:20AM,253452,19,"AdvaGen Pharma, Ltd",48
2,Dec 19 2022 4:58PM,253449,15,"Brookfield Pharmaceuticals, LLC",30
3,Dec 19 2022 4:56PM,253448,15,"Virtus Pharmaceuticals, LLC",1
4,Dec 19 2022 4:55PM,253446,15,"Alliance Pharma, Inc.",15
5,Dec 19 2022 4:50PM,253445,10,"Snap Medical Industries, LLC",2
6,Dec 19 2022 4:48PM,253444,22,"NBTY Global, Inc.",1
7,Dec 19 2022 4:47PM,253442,10,ISDIN Corporation,13
8,Dec 19 2022 4:47PM,253441,10,ISDIN Corporation,10
9,Dec 19 2022 4:39PM,253440,10,ISDIN Corporation,4


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Dec 20 2022 8:28AM,253453,19,ACG North America LLC,1
1,Dec 20 2022 8:20AM,253452,19,"AdvaGen Pharma, Ltd",48
2,Dec 19 2022 4:58PM,253449,15,"Brookfield Pharmaceuticals, LLC",30
3,Dec 19 2022 4:56PM,253448,15,"Virtus Pharmaceuticals, LLC",1
4,Dec 19 2022 4:55PM,253446,15,"Alliance Pharma, Inc.",15


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet
Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released
0,Dec 20 2022 8:28AM,253453,19,ACG North America LLC,1
1,Dec 20 2022 8:20AM,253452,19,"AdvaGen Pharma, Ltd",48
2,Dec 19 2022 4:58PM,253449,15,"Brookfield Pharmaceuticals, LLC",30
3,Dec 19 2022 4:56PM,253448,15,"Virtus Pharmaceuticals, LLC",1
4,Dec 19 2022 4:55PM,253446,15,"Alliance Pharma, Inc.",15


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Dec 20 2022 8:28AM,253453,19,ACG North America LLC,1
1,Dec 20 2022 8:20AM,253452,19,"AdvaGen Pharma, Ltd",48
2,Dec 19 2022 4:58PM,253449,15,"Brookfield Pharmaceuticals, LLC",30
3,Dec 19 2022 4:56PM,253448,15,"Virtus Pharmaceuticals, LLC",1
4,Dec 19 2022 4:55PM,253446,15,"Alliance Pharma, Inc.",15


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released
Warehouse,,
10,3041014,55
15,1774010,56
19,1267133,56
21,1013590,4
22,253444,1


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released
0,10,3041014,55
1,15,1774010,56
2,19,1267133,56
3,21,1013590,4
4,22,253444,1


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released
0,10,55
1,15,56
2,19,56
3,21,4
4,22,1


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,55
1,15,Released,56
2,19,Released,56
3,21,Released,4
4,22,Released,1


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,21,22
Status,,,,,
Released,55,56,56,4,1


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,21,22
0,Released,55,56,56,4,1


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,21,22
0,Released,55,56,56,4,1


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()